In [18]:
text_file = open("20150620_13_59_09_349_0_old", "r")
lines = text_file.read().split('\n')
print(len(lines))

1000004


In [99]:
currents = [line for line in lines if '3 0x11' in line]
voltages = [line for line in lines if '3 0x13' in line]
print(len(currents), len(voltages), '\n-------', currents[0:10], '\n-------', voltages[0:10])

10297 10558 
------- ['13:59:14.292 3 0x11 4 [ 172, 162, 186, 65] ', '13:59:14.316 3 0x11 4 [ 29, 48, 33, 66] ', '13:59:14.332 3 0x11 4 [ 154, 91, 35, 66] ', '13:59:14.356 3 0x11 4 [ 13, 125, 29, 66] ', '13:59:14.364 3 0x11 4 [ 0, 0, 0, 0] ', '13:59:14.388 3 0x11 4 [ 0, 0, 0, 0] ', '13:59:14.396 3 0x11 4 [ 216, 70, 176, 63] ', '13:59:14.404 3 0x11 4 [ 198, 32, 61, 65] ', '13:59:14.412 3 0x11 4 [ 109, 217, 123, 65] ', '13:59:14.412 3 0x11 4 [ 151, 6, 128, 65] '] 
------- ['13:59:14.292 3 0x13 4 [ 227, 17, 148, 67] ', '13:59:14.308 3 0x13 4 [ 221, 206, 147, 67] ', '13:59:14.316 3 0x13 4 [ 134, 129, 147, 67] ', '13:59:14.332 3 0x13 4 [ 231, 108, 147, 67] ', '13:59:14.340 3 0x13 4 [ 94, 124, 147, 67] ', '13:59:14.356 3 0x13 4 [ 94, 124, 147, 67] ', '13:59:14.364 3 0x13 4 [ 208, 213, 148, 67] ', '13:59:14.388 3 0x13 4 [ 111, 234, 148, 67] ', '13:59:14.396 3 0x13 4 [ 32, 224, 148, 67] ', '13:59:14.404 3 0x13 4 [ 122, 136, 148, 67] ']


In [101]:
import struct
import sys
import datetime

start_time = ''

current_times = []
for i in range(len(currents)):
    bytes = currents[i].split('[')[1][1: -2].split(', ')
    hex = ''
    bytes.reverse()
    for byte in bytes:
        hex += bin(int(byte))[2:].zfill(8)
    f = int(hex, 2)
    f = struct.unpack('f', struct.pack('I', f))[0]
    
    s = currents[i].split(' ')[0]
    if i == 0:
        start_time = datetime.datetime(2018, 9, 8, int(s[0:2]), int(s[3:5]), int(s[6:8]), int('{:<06d}'.format(int(s[9:]))))
        current_times.append([0.0 / 3600.0, f])
    else:
        current_times.append([datetime.timedelta.total_seconds(
            datetime.datetime(2018, 9, 8, int(s[0:2]), int(s[3:5]), int(s[6:8]), int('{:<06d}'.format(int(s[9:])))) -
            start_time
        ) / 3600, f])
print(len(current_times), current_times[0:10])

voltage_times = []
for i in range(len(voltages)):
    bytes = voltages[i].split('[')[1][1: -2].split(', ')
    hex = ''
    bytes.reverse()
    for byte in bytes:
        hex += bin(int(byte))[2:].zfill(8)
    f = int(hex, 2)
    f = struct.unpack('f', struct.pack('I', f))[0]
    
    s = voltages[i].split(' ')[0]
    if i == 0:
        start_time = datetime.datetime(2018, 9, 8, int(s[0:2]), int(s[3:5]), int(s[6:8]), int('{:<06d}'.format(int(s[9:]))))
        voltage_times.append([0.0 / 3600.0, f])
    else:
        voltage_times.append([datetime.timedelta.total_seconds(
            datetime.datetime(2018, 9, 8, int(s[0:2]), int(s[3:5]), int(s[6:8]), int('{:<06d}'.format(int(s[9:])))) -
            start_time
        ) / 3600, f])
print(len(current_times), len(voltage_times), '\n-------', current_times[0:10], '\n-------', voltage_times[0:10])

10297 [[0.0, 23.329429626464844], [6.666666666666667e-06, 40.2969856262207], [1.1111111111111112e-05, 40.839454650878906], [1.7777777777777777e-05, 39.37211990356445], [1.9999999999999998e-05, 0.0], [2.6666666666666667e-05, 0.0], [2.8888888888888888e-05, 1.377161979675293], [3.111111111111111e-05, 11.820501327514648], [3.3333333333333335e-05, 15.740582466125488], [3.3333333333333335e-05, 16.003217697143555]]
10297 10558 
------- [[0.0, 23.329429626464844], [6.666666666666667e-06, 40.2969856262207], [1.1111111111111112e-05, 40.839454650878906], [1.7777777777777777e-05, 39.37211990356445], [1.9999999999999998e-05, 0.0], [2.6666666666666667e-05, 0.0], [2.8888888888888888e-05, 1.377161979675293], [3.111111111111111e-05, 11.820501327514648], [3.3333333333333335e-05, 15.740582466125488], [3.3333333333333335e-05, 16.003217697143555]] 
------- [[0.0, 296.1397399902344], [4.444444444444444e-06, 295.6161193847656], [6.666666666666667e-06, 295.01190185546875], [1.1111111111111112e-05, 294.8507995

In [107]:
powers = []


common_times = list(set(value[0] for value in current_times if value[0] in [val[0] for val in voltage_times]))


good_current_times = []
for i in range(0, len(current_times)):
    if current_times[i][0] in common_times and current_times[i][0] not in [val[0] for val in good_current_times]:
        good_current_times.append(current_times[i])

good_voltage_times = []
for i in range(0, len(voltage_times)):
    if voltage_times[i][0] in common_times and voltage_times[i][0] not in [val[0] for val in good_voltage_times]:
        good_voltage_times.append(voltage_times[i])
print(len(good_current_times), len(good_voltage_times))


for i in range(len(good_current_times)):
    powers.append([good_current_times[i][0], good_current_times[i][1] * good_voltage_times[i][1]])

print(len(powers), powers[0:10])

9713 9713
9713 [[0.0, 6908.7712237017695], [6.666666666666667e-06, 11888.090368633857], [1.1111111111111112e-05, 12041.54585742834], [1.7777777777777777e-05, 11613.657937728567], [1.9999999999999998e-05, 0.0], [2.6666666666666667e-05, 0.0], [2.8888888888888888e-05, 410.05132433306426], [3.111111111111111e-05, 3511.4716850396944], [3.3333333333333335e-05, 4670.92273466062], [4.222222222222222e-05, 4459.667232917796]]


In [108]:
total_energy = 0.0
total_energy += powers[0][0] * powers[0][1] / 1000
for i in range(2, len(powers)):
    total_energy += (powers[i][0] - powers[i-1][0]) *  powers[i][1] / 1000
    # integrate with reinman sum
print(total_energy)

4.663028381891487


velocity = angular velocity * radius, average speed take integral